<h1>SI 608 Final Project</h1>

In [1]:
import pandas as pd
import os
import networkx as nx
from networkx.algorithms import bipartite, hits

from utils import * 

In [2]:
users = load_chunks("../VoterFraud2020-main/data/users")
retweets = load_chunks("../VoterFraud2020-main/data/retweets")
tweets = load_chunks("../VoterFraud2020-main/data/tweets")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte

<h2>Construct retweet network</h2>

In [ ]:
retweets_to_users = retweets.merge(users[['user_id', 'user_active_status']], on = 'user_id')
retweets_to_users = retweets_to_users.rename(columns = {'user_id': 'rt_user_id'})

In [ ]:
retweets.columns = ['tweet_id', 'rting_user_id']

In [ ]:
tweets = tweets[(tweets['retweet_count_metadata'] > 0) | (tweets['quote_count_metadata'] > 0)]

In [ ]:
retweets_to_tweets = tweets.merge(retweets, on = 'tweet_id', how = 'inner')

In [ ]:
#prototyping subset 
retweets_to_tweets = retweets_to_tweets.sample(100000)

In [ ]:
G = nx.Graph()
G.add_nodes_from(retweets_to_tweets['tweet_id'], bipartite = 1)
G.add_nodes_from(retweets_to_tweets['rting_user_id'].astype(str), bipartite = 0)

G.add_edges_from([(row['rting_user_id'], row['tweet_id']) for idx, row in retweets_to_tweets.iterrows()])

In [ ]:
nx.is_connected(G)

<h2>HITS Algorithm</h2>

In [ ]:
h, a = nx.hits(G, max_iter = 500)

In [ ]:
hubs = pd.DataFrame(h.items())
hubs.columns = ['user_id', 'hub_score']

authorities = pd.DataFrame(a.items())
authorities.columns = ['tweet_id', 'authority_score']

In [ ]:
authorities.sort_values(by = 'authority_score', ascending = False)

<h3>Betweeness Centrality</h3>

In [ ]:
bottom_nodes, top_nodes = bipartite.sets(G)
#nx.bipartite.betweenness_centrality(G, nodes=top_nodes)